In [105]:
import json

def get_key():
    with open("secretkey.json", "r") as json_file:
        return json.load(json_file)["api_key"]

In [106]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor


def remove_all_placeholders(slide):
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)


def add_title_slide(prs, title_text, subtitle_text):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    remove_all_placeholders(slide)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add title text box
    title_shape = slide.shapes.add_textbox(
        Inches(1), Inches(2), slide_width - Inches(2), Inches(1.5)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = Pt(54)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add subtitle text box
    subtitle_shape = slide.shapes.add_textbox(
        Inches(1), Inches(4), slide_width - Inches(2), Inches(1)
    )
    subtitle_frame = subtitle_shape.text_frame
    subtitle_frame.text = subtitle_text
    subtitle_frame.paragraphs[0].font.size = Pt(32)
    subtitle_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    subtitle_frame.paragraphs[0].font.color.rgb = RGBColor(105, 105, 105) 


def add_table_of_contents_slide(prs, title_text, num_sections, sections):
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    remove_all_placeholders(slide)

    title_shape = slide.shapes.add_textbox(
        Inches(0.5), Inches(0.5), Inches(12.33), Inches(1)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = Pt(32)
    title_frame.paragraphs[0].font.bold = True

    # Constants for positioning
    left_positions = [Inches(0.5), Inches(6.5)]  # Adjusted to move more to the left
    top_start = Inches(1.5)
    space_between = Inches(1.5)
    max_per_column = 4

    # Function to adjust text size if overflowing
    def adjust_text_size(text_frame, max_width):
        for paragraph in text_frame.paragraphs:
            run = paragraph.runs[0]
            while (
                run.font.size is None or run.font.size.pt > 10
            ):  # Set a minimum font size
                if run.font.size is not None and text_frame.auto_size:
                    break
                if run.font.size is None:
                    run.font.size = Pt(20)  # Start with 20 if size is None
                else:
                    run.font.size -= Pt(1)  # Decrease font size by 1pt

    for i, (heading, subheading) in enumerate(sections):
        column = i // max_per_column
        row = i % max_per_column

        # Calculate positions
        left = left_positions[column]
        top = top_start + row * space_between

        # Add section number
        number_shape = slide.shapes.add_textbox(left, top, Inches(1.0), Inches(1.0))
        number_frame = number_shape.text_frame
        number_frame.text = f"{i + 1:02}"
        number_frame.paragraphs[0].font.size = Pt(40)
        number_frame.paragraphs[0].font.bold = True
        number_frame.paragraphs[0].font.color.rgb = RGBColor(89, 89, 89)

        # Add section heading
        heading_shape = slide.shapes.add_textbox(
            left + Inches(1.0),
            top,
            Inches(5.0),
            Inches(0.5),  # Moved closer to the left
        )
        heading_frame = heading_shape.text_frame
        heading_frame.text = heading
        heading_frame.paragraphs[0].font.size = Pt(20)
        heading_frame.paragraphs[0].font.bold = True
        adjust_text_size(heading_frame, Inches(5.0))

        # Add section subheading with wrapping text
        subheading_shape = slide.shapes.add_textbox(
            left + Inches(1.0),
            top + Inches(0.5),
            Inches(5.0),
            Inches(1.0),  # Moved closer to the left
        )
        subheading_frame = subheading_shape.text_frame
        subheading_frame.text = subheading
        subheading_frame.word_wrap = True  # Enable word wrapping
        subheading_frame.paragraphs[0].font.size = Pt(14)
        subheading_frame.paragraphs[0].font.color.rgb = RGBColor(128, 128, 128)
        adjust_text_size(subheading_frame, Inches(5.0))


def add_section_slide(prs, section_number, heading, subheading):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    remove_all_placeholders(slide)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add section number
    number_shape = slide.shapes.add_textbox(
        Inches(6.0), Inches(2), Inches(1.0), Inches(1.0)
    )
    number_frame = number_shape.text_frame
    number_frame.text = f"{section_number:02}"
    number_frame.paragraphs[0].font.size = Pt(100)
    number_frame.paragraphs[0].font.bold = True
    number_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add heading text box
    heading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(3.5), slide_width - Inches(2), Inches(1.5)
    )
    heading_frame = heading_shape.text_frame
    heading_frame.text = heading
    heading_frame.paragraphs[0].font.size = Pt(54)
    heading_frame.paragraphs[0].font.bold = True
    heading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add subheading text box
    subheading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(5), slide_width - Inches(2), Inches(1)
    )
    subheading_frame = subheading_shape.text_frame
    subheading_frame.text = subheading
    subheading_frame.paragraphs[0].font.size = Pt(32)
    subheading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    subheading_frame.paragraphs[0].font.color.rgb = RGBColor(105, 105, 105)


def main():
    prs = Presentation()

    # Set slide width and height for 16:9 aspect ratio
    prs.slide_width = Inches(13.33)
    prs.slide_height = Inches(7.5)

    # Add a title slide
    add_title_slide(prs, "Hello, Pookie😘", "This is a subtitle in your first slide")

    sections = [
        ("Name of the Section 1", "You can describe the topic of the section here"),
        ("Name of the Section 2", "You can describe the topic of the section here"),
        ("Name of the Section 3", "You can describe the topic of the section here"),
        ("Name of the Section 4", "You can describe the topic of the section here"),
        ("Name of the Section 5", "You can describe the topic of the section here"),
        ("Name of the Section 6", "You can describe the topic of the section here"),
        ("Name of the Section 7", "You can describe the topic of the section here"),
        ("Name of the Section 8", "You can describe the topic of the section here"),
    ]

    # Add a table of contents slide
    add_table_of_contents_slide(prs, "Table of Contents", len(sections), sections)

    for i, (heading, subheading) in enumerate(sections):
        add_section_slide(prs, i + 1, heading, subheading)

    # Save the presentation
    prs.save("my_presentation.pptx")

    print("Presentation created successfully")


if __name__ == "__main__":
    main()

Presentation created successfully
